In [2]:
# import sys
# print(sys.executable)

C:\Users\Admin\Jupyter\Posture Detection System\posture_env\Scripts\python.exe


In [ ]:
# !pip install scikit-learn pandas 
# !pip install mediapipe opencv-python

### Setup Mediapipe

In [2]:
import mediapipe as mp
import cv2
print("Mediapipe is working! Version:", mp.__version__)
print("Computer Vision is working! Version:", cv2.__version__)

Mediapipe is working! Version: 0.10.21
Computer Vision is working! Version: 4.11.0


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Get Realtime Webcam Feed

In [4]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    if not ret: 
        break
        
    cv2.imshow("Raw Webcam Feed",frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Make Detections from Feed

In [18]:
cap = cv2.VideoCapture(0)

# Initialize pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make detections
    results = pose.process(image)

    # Recolor back to BGR for rendering 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose Detection', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
results.pose_landmarks #.landmark[0].visibility

x: 0.873660266
y: 0.30303818
z: -0.870196342
visibility: 0.999193132

# Capture Landmarks & Export to CSV
<!-- <img src="https://i.imgur.com/8bForKY.png"> -->
<!-- <img src="https://i.imgur.com/AzKNp7A.png"> -->

In [20]:
import csv
import os
import numpy as np
import time

In [13]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

In [14]:
landmarks =["class"]
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [16]:
landmarks[-1]

'v33'

### Writing to CSV 

In [17]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Capturing Landmarks for classes

In [ ]:
class_name = "Upright"  #"Upright"," "Slouching," and "Leaning Forward"

In [ ]:
cap = cv2.VideoCapture(0)

# Initialize pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make detections
    results = pose.process(image)

    # Recolor back to BGR for rendering 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    try:
        # Extract Pose Lnadmarks
        pose =  results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
        
        row = pose_row

        #Append class name
        row.insert(0,class_name)
        
        # Export to CSV
        with open('coords.csv', mode='w', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
                
    except:
        pass
    
    
    cv2.imshow('Pose Detection', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Train Custom Model Using Scikit Learn